# Оцінка моделей

In [58]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from joblib import load

Зчитування файлу processed_data.csv.

In [59]:
df = pd.read_csv("../data/processed/processed_data.csv")

In [60]:
X = df[df.columns[1:-1]]
y = df[df.columns[-1:]].values.flatten()

__Функція для виведення класифікаційного звіту__

In [61]:
def model_classification_report(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    print(
        f"Класифікаційний звіт для моделі {model_name}:\n",
        classification_report(y_test, y_pred),
    )

__Функція для виведення розміру моделі__

In [62]:
def model_size(model_path: str):
    size_in_bytes = os.path.getsize(model_path)
    size_in_kb = size_in_bytes / 1024
    size_in_mb = size_in_kb / 1024
    print(
        f"Розмір моделі: {size_in_bytes} B ({size_in_kb:.2f} kB / {size_in_mb:.2f} MB)"
    )

Завантаження моделей.

In [63]:
model_RF_path = "../models/model_RandomForest.joblib"
model_HGB_path = "../models/model_HistGradientBoosting.joblib"
model_GB_path = "../models/model_GradientBoosting.joblib"
model_LR_path = "../models/model_LogisticRegression.joblib"

In [65]:
model_RF = load(model_RF_path)
model_HGB = load(model_HGB_path)
model_GB = load(model_GB_path)
model_LR = load(model_LR_path)

__Модель Random Forest__

In [67]:
model_classification_report(model_RF, "Random Forest", X, y)
model_size(model_RF_path)

Класифікаційний звіт для моделі Random Forest:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98     32224
           1       0.98      0.98      0.98     40050

    accuracy                           0.98     72274
   macro avg       0.98      0.98      0.98     72274
weighted avg       0.98      0.98      0.98     72274

Розмір моделі: 57228153 B (55886.87 kB / 54.58 MB)


__Модель Histogram-based Gradient Boosting Classification Tree__

In [69]:
model_classification_report(
    model_HGB, "Histogram-based Gradient Boosting Classification Tree", X, y
)
model_size(model_HGB_path)

Класифікаційний звіт для моделі Histogram-based Gradient Boosting Classification Tree:
               precision    recall  f1-score   support

           0       0.95      0.96      0.95     32224
           1       0.97      0.96      0.96     40050

    accuracy                           0.96     72274
   macro avg       0.96      0.96      0.96     72274
weighted avg       0.96      0.96      0.96     72274

Розмір моделі: 1976064 B (1929.75 kB / 1.88 MB)


__Модель Gradient Boosting__

In [71]:
model_classification_report(model_GB, "Gradient Boosting", X, y)
model_size(model_GB_path)

Класифікаційний звіт для моделі Gradient Boosting:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94     32224
           1       0.96      0.94      0.95     40050

    accuracy                           0.95     72274
   macro avg       0.95      0.95      0.95     72274
weighted avg       0.95      0.95      0.95     72274

Розмір моделі: 924237 B (902.58 kB / 0.88 MB)


__Модель Logistic Regression__

In [73]:
model_classification_report(model_LR, "Logistic Regression", X, y)
model_size(model_LR_path)

Класифікаційний звіт для моделі Logistic Regression:
               precision    recall  f1-score   support

           0       0.84      0.77      0.81     32224
           1       0.83      0.88      0.86     40050

    accuracy                           0.83     72274
   macro avg       0.84      0.83      0.83     72274
weighted avg       0.83      0.83      0.83     72274

Розмір моделі: 1439 B (1.41 kB / 0.00 MB)
